# Script de estimación volumétrica a través de modelos de ahusamiento

**Metodos usados para la obtencion de los parametros:**

1.Algoritmo de Gauss-Newton

2.Algoritmo de Levenberg-Marquardt

#El sistema de R soluciona por defecto el problema de minimos cuadrados no lineales a traves del algoritmo de Gauss-Newton (nls)

#Para instalar la libreria del algoritmo de Levenberg-Marquardt solo se ejecutar la siguiente linea de codigo: 

install.packages ("minpack.lm")
 
#Una vez instalada solo queda importarla si es necesario 

library (minpack.lm) 

# Inicio del Script

In [ ]:
library (readxl)
library (minpack.lm)

In [ ]:
#Apagar la notacion cientifica 
options (scipen = 999)

* Numero de datos ajustados en los modelos, ingresar el numero total de individuos, a modo de ejemplo se ajustara sobre 73 individuos \ de _Eucalyptus globulus_

In [ ]:
total_individuos <- 73

In [ ]:
n = ((total_individuos*70)/100)

In [ ]:
paste ("El numero de individuos sobre el cual se debe realizar el ajuste es de: "
, trunc(n))

# Cociente de esbeltez, relacion altura y diametro con el fin de observar la regularidad de los individuos

In [ ]:
tabla <- read_excel("C:/Users/Administrador/Dropbox/proyecto grado/DatosEucalyptus.xlsx",
                    sheet = 1, col_names = TRUE, col_types = NULL, na = "", skip = 0) 
tabla$esbeltez <- tabla$H/tabla$DAP #Cociente de esbeltez
head (tabla)

# Llamando la base de datos a traves de la libreria readxl sobre el 70% del total de los individuos (haciendo el ajuste), hoja numero dos en Excel..

In [ ]:
tabla2 <- read_excel("C:/Users/Administrador/Dropbox/proyecto grado/DatosEucalyptus.xlsx", sheet=2,
                     col_names = TRUE, col_types = NULL, na = "", skip = 0) 
tabla2
#View (tabla)

#------------------------------------------------------------------------------------

#Las variables invocadas de la base de datos en la hoja de calculo deben tener la misma escrituta, por ejemplo, si deseo llamar a la variable Y en R esta debe estar declarada de la misma manera ya sea (y minuscula o Y MAYUSCULA), de esta manera no se generaran posibles errores por objetos no encontrados la base de datos original

# Ajustando los modelos 

Metodos de convergencia empleados, **A.** Algoritmo de Gauss Newton y **B.** Algortimo de Levenbert Marquardt:

\begin{equation}\label{eq:ej}
A. x_{+} = x_{c} (J(x_{c})^{T} J(x_{c})^{-1} J(x_{c})^{T} R(x_{c}) \\
B. x_{+} = x_{c} (J(x_{c})^{T} J(x_{c} + μcI)^{-1} J(x_{c})^{T} R(x_{c})
\end{equation}

# Modelo de ahusamiento propuesto por Bruce, Curtis & Vancoevering (1968)

In [ ]:
RnlinealB <- Y ~ ((b1*X^1.5) + (b2*X^3) + (b3*X^32))
nlmodB <- nls (RnlinealB, data = tabla2, start = list(b1 = 0.1, b2 = 0.1, b3 = 0.1))
summary (nlmodB)
coe <- coef (nlmodB)
b1 = coe [1]
b1
b2 = coe [2]
b2
b3 = coe [3]
b3

# Modelo de ahusamiento propuesto por Kozak, Munro, & Smith (1969)  

Ilustracion del modelo de ahusamiento de Kozak, Munro, & Smith (1969)

\begin{equation}\label{eq:ej}
Y = b_{1} (T-1) + b_{2} (T^{2} - 1) 
\end{equation}

In [ ]:
# Modelo de ahusamiento propuesto por Kozak, Munro, & Smith (1969)     

RnlinealK <- Y ~ (b1*(T - 1) + b2*(T^2 - 1))
nlmodK <- nls (RnlinealK, data = tabla2, start = list(b1 = 0.1, b2 = 0.1))
summary (nlmodK) 
coe <- coef (nlmodK)
b1 = coe [1]
b1 
b2 = coe [2]
b2

# Ajustando el volumen sobre el 30% de los datos, fase de validacion, hoja numero tres, sheet=3

In [ ]:
tabla3 <- read_excel("C:/Users/Administrador/Dropbox/proyecto grado/DatosEucalyptus.xlsx",
                     sheet = 3, col_names = TRUE, col_types = NULL, na = "", skip = 0) 
head (tabla3)

# Estimacion del volumen por metodos numericos (Integrar modelo de Kozak para todos los individuos)

\begin{equation}
V = \int_{0}^{ht} \pi/40000*DSC^{2}*P(x) \cdot dx
\end{equation}

In [ ]:
for (i in 1:nrow (tabla3)){
 dsc <- tabla3$DSC [i]      
 ht <- tabla3$H [i]
 a = pi/40000
 k <- a*(dsc^2)
 fk <- function(h) k*(b1*((h/ht)-1)+b2*((h/ht)^2-1))  
 volkozak <- integrate (fk,0,ht) 
 print(volkozak$value)
}

# Estimacion del volumen por metodos analiticos  

Para la estimación del volumen observado se trató el individuo como un cilindro desde la zona
del tocón hasta el DAP, luego a través de la ecuación de Smalian desde la zona del DAP hasta la
altura total del
individuo como un paraboloide, obteniendo asi:

\begin{equation}
V = (\pi/40000*DSC^{2}*1.3) + \pi/40000*(\frac{DSC^{2} + Di^{2}}{2})*L 
\end{equation}

In [ ]:
# El metodo analitico tomara como forma del arbol un cilindro desde la zona del tocon hasta el diametro a la altura del pecho, y un paraboloide (Smalian) desde el dap hasta la altura total del individuo.

#Volumenes (Observados vs predichos), sobre el 30% de los datos, hoja numero 3

a = pi/40000
di = 0.01 #tomado como diametro inferior de aproximadamente 1 cm
l = (tabla3$H-1.3) #longitud fustal para estimacion del paraboloide
tabla3$vol_metanalitico <- (pi/40000 * ((tabla3$DSC)^2) * 1.3) + (a*((tabla3$DSC^2+di^2)/2)*l)
vobs <- tabla3$vol_metanalitico
vobs

# Coeficiente de determinación, correlacion entre el volumen observado (Metodo analitico) y volumen predicho (volumen ajustado a traves de los modelos de ahusamiento)


In [ ]:
rls = lm (volkozak ~ vobs, data = tabla3)
summary(rls)#muestra las caracteristicas estadisticas del ajuste del modelo
coef(rls)#evaluar los coef del modelo
residuals (rls) #Invocando los residuales del modelo de regresion

# Obteniendo el coeficiente de determinacion,los valores mas cercanos al 1 demostraran una mayor aptitud del modelo 
rlsc = summary (rls)
r2 = rlsc [8]
r2 

# Graficos para observar el cumplimiento de los supuestos: 

In [ ]:
#1. Aleatoriedad de los residuales 
#2. Distribucion normal 
#3. Homocedasticidad de varianza
#4. Transformacion de cook 

win.graph()
par (mfrow = c(2,2)) #Muestra en una ventana el grafico de los supuestos
plot (lm(volkozak ~ vobs, data = tabla3))

#xlab = "Volumen Observado", ylab = "Residuales", main = "Cumplimiento de supuestos"

win.graph()
par(mfrow = c(2, 2))
rls <- lm(volkozak ~ vobs, data = tabla3)
plot(residuals(rls) ~ tabla3$volkozak, xlab = "Volumen Predicho", 
     ylab = "Residuales", main = "Aleatoriedad de los residuales", col = "blue")
abline(h = 0, lty = 2)

# Calculo de la raiz del error cuadratico medio RMSE

In [ ]:
#n = numero de datos ajustados en el modelo

RMSE = ((sum(residuals(rls)^2))/(n-2))^1/2
paste ("La raiz del errror cuadratico medio es: ", trunc(RMSE))

# Modelo de ahusamiento propuesto por Demaerschalk (1972)  

In [ ]:
RnlinealD72 <- Y ~ (b1 * (Z^b2))
nlmodD72 <- nls (RnlinealD72, data = tabla2, start = list(b1 = 0.1, b2 = 0.1))
summary (nlmodD72)
coe <- coef (nlmodD72)
b1 = coe [1]
b1
b2 = coe [2]
b2

# Ajustando el volumen sobre el 30% de los datos, fase de validacion, hoja numero tres, sheet=3

In [ ]:
tabla3 <- read_excel("C:/Users/Administrador/Dropbox/proyecto grado/DatosEucalyptus.xlsx",sheet=3,col_names=TRUE,col_types=NULL,na="", skip=0) 
head (tabla3)

# Integrar modelo de Demaerschalk (1972) para todos los individuos

In [ ]:
for (i in 1:nrow (tabla3)){
 dsc <- tabla3$DSC [i]      
 ht <- tabla3$H [i]
 a <- pi/40000
 k <- a*(dsc^2)
 fd72 = function(h) k*(b1*((ht-h)/(ht))^b2)  
 voldemaerschalk <- integrate (fd72,0,ht) 
 print(voldemaerschalk$value)
}

# Estimacion del volumen por metodos analiticos

In [ ]:
# El metodo analitico tomara como forma del arbol un cilindro desde la zona del tocon hasta el diametro a la altura del pecho, y un paraboloide (Smalian) desde el dap hasta la altura total del individuo.

#Volumenes (Observados vs predichos), sobre el 30% de los datos, hoja numero 3

a = pi/40000
di = 0.01 #tomado como diametro inferior de aproximadamente 1 cm
l = (tabla3$H-1.3) #longitud fustal para estimacion del paraboloide
tabla3$vol_metanalitico <- (pi/40000 * ((tabla3$DSC)^2) * 1.3) + (a*((tabla3$DSC^2+di^2)/2)*l)
vobs <- tabla3$vol_metanalitico
vobs

# Regresion lineal simple, entre el volumen observado (Metodo analitico) y volumen predicho (volumen ajustado a traves de los modelos de ahusamiento)

In [ ]:
rls = lm (voldemaerschalk ~ vobs, data = tabla3)
summary(rls)#muestra las caracteristicas estadisticas del ajuste del modelo
coef(rls)#evaluar los coef del modelo
residuals (rls) #Invocando los residuales del modelo de regresion

# Obteniendo el coeficiente de determinacion,los valores mas cercanos al 1 demostraran una mayor aptitud del modelo 
rlsc = summary (rls)
r2 = rlsc [8]
r2 

# Graficos para observar el cumplimiento de los supuestos:

In [ ]:
#1. Aleatoriedad de los residuales 
#2. Distribucion normal 
#3. Homocedasticidad de varianza
#4. Transformacion de cook 

win.graph()
par (mfrow = c(2,2)) #Muestra en una ventana el grafico de los supuestos
plot (lm(voldemaerschalk ~ vobs, data = tabla3))

win.graph()
par(mfrow = c(2, 2))
rls <- lm (voldemaerschalk ~ vobs, data = tabla3)
plot(residuals(rls)  ~ tabla3$voldemaerschalk, xlab = "Volumen Predicho", ylab = "Residuales", main = "Aleatoriedad de los residuales", col = "blue")
abline(h = 0, lty = 2)

# Calculo de la raiz del error cuadratico medio RMSE

In [ ]:
#n = numero de datos ajustados en el modelo

RMSE = ((sum(residuals(rls)^2))/(n-2))^1/2
paste ("La raiz del errror cuadratico medio es: ", trunc(RMSE))

# Modelo de ahusamiento) propuesto por Demaerschalk (1973) 

In [ ]:
RnlinealD <- Y ~ ((b1 / DAP^2 * H) * Z^b2 + b3 * Z^b4)
nlmodD <- nls (RnlinealD, data = tabla2, start = list(b1 = 0.1, b2 = 0.1, b3 = 0.1, b4 = 0.1))
summary (nlmodD)
coe <- coef (nlmodD) 
b1 = coe [1]
b1
b2 = coe [2]
b2
b3 = coe [3]
b3
b4 = coe [4]
b4

# Como no hubo convergencia de parametros por el metodo de Newton, se empleará el método de Levenbert-Mardquardt para obeservar si hay convergencia o no

In [ ]:
#2

RnlinealD <- Y ~ ((b1 / DAP^2 * H) * Z^b2 + b3 * Z^b4)
LMD <- nlsLM (RnlinealD, data = tabla2, start = list(b1 = 0.1, b2 = 0.1, b3 = 0.1, b4 = 0.1))
summary (LMD) 
coe <- coef (LMD)
b1 = coe [1]
b1
b2 = coe [2]
b2
b3 = coe [3]
b3
b4 = coe [4]
b4

# Modelo de ahusamiento propuesto por Lowell (1986)

In [ ]:
RnlinealL <- Y ~ ((b1*X) + (b2*X^2) + (b3*X^3) + (b4*X^4) + (b5*X^5))
nlmodL <- nls (RnlinealL, data = tabla2, start = list(b1 = 0.1, b2 = 0.1, b3 = 0.1, b4 = 0.1, b5 = 0.1))
summary (nlmodL)
coe <- coef (nlmodL)
b1 = coe [1]
b1
b2 = coe [2]
b2
b3 = coe [3]
b3
b4 = coe [4]
b4
b5 = coe [5]
b5

# Modelo de ahusamiento trigonometrico propuesto por Thomas y Parresol (1991)

In [ ]:
#Argumentos expresados en radianes

RtTP <- Y ~ ((b1*T-1) + b2*sin(2*pi*T) + b3*(1/tan(pi/2*T))) 
modtTP <- nls (RtTP, data = tabla2, start = list (b1 = 0.1, b2 = 0.1, b3 = 0.1))
summary (modtTP)
coe <- coef (modtTP)
b1 = coe [1]
b1
b2 = coe [2]
b2
b3 = coe [3]
b3

# Modelo de ahusamiento propuesto por Renteria & Ramirez (1998)

In [ ]:
RnlinealR <- Y ~ ((b1*Z) + (b2*Z^2) + (b3*Z^3))
nlmodR <- nls (RnlinealR, data = tabla2, start = list(b1 = 0.1, b2 = 0.1, b3=0.1))
summary (nlmodR)
coe <- coef (nlmodR)
b1 = coe [1]
b1
b2 = coe [2]
b2
b3 = coe [3]
b3

# Modelo de ahusamiento propuesto por Coffre (Rojo et al. (2005)

In [ ]:
RnlinealC <- Y ~ ((b1*X) + (b2*X^2) + (b3*X^3))
nlmodC <- nls (RnlinealC, data = tabla2, start = list(b1 = 0.1, b2 = 0.1, b3 = 0.1))
summary (nlmodC)
coe <- coef (nlmodC)
b1 = coe [1]
b1
b2 = coe [2]
b2
b3 = coe [3]
b3

# Modelo de ahusamiento propuesto por J. Andres Rodriguez Toro et al (2016)

In [ ]:
RnlinealT <- Y ~ X/(b0+b1*X+b2*X^2+b3*X^3)
nlmodT <- nls (RnlinealT, data = tabla2, start = list(b0 = 0.1, b1 = 0.1, b2 = 0.1, b3 = 0.1))
summary (nlmodT) 
coe <- coef (nlmodT)
b0 = coe [1]
b0
b1 = coe [2]
b1
b2 = coe [3] 
b2
b3 = coe [4]
b3

# Indice de Criterio Akaike

\begin{equation}\label{eq:ej}
AIC = 2logLik + 2K
\end{equation}

In [ ]:

#Al comparar modelos ajustados por la máxima probabilidad a los mismos datos, cuanto más pequeño sea el AIC o BIC, mejor será el ajuste.

#?AIC

IAB = AIC (nlmodB)

IAK = AIC (nlmodK)

IAD72 = AIC (nlmodD72) 

IAD73 = AIC (LMD)

IAL = AIC (nlmodL)

IATP = AIC (modtTP)

IAR = AIC (nlmodR)
 
IAC = AIC (nlmodC)

IAT = AIC (nlmodT)

#Estructura de control para seleccion del criterio para Akaike en base al ajuste de los modelos

if ((IAR)<(IAK) && (IAR)<(IAB) && (IAR)<(IAD72) && (IAR)<(IATP)) {
  print ("El modelo mejor ajustado por el indice de Akaike es el de: Renteria")
} else if ((IAK)<(IAR) && (IAK)<(IAB) && (IAK)<(IAD72) && (IAK)<(IATP)) {
  print ("El modelo mejor ajustado por el indice de Akaike es el de: Kozak")
} else if ((IAB)<(IAR) && (IAB)<(IAK) && (IAB)<(IAD72) && (IAB)<(IATP)) {
  print ("El modelo mejor ajustado por el indice de Akaike es el de: Bruce")
} else if ((IAD72)<(IAR) && (IAD72)<(IAK) && (IAD72)<(IAB) && (IAD72)<(IATP)) {
  print ("El modelo mejor ajustado por el indice de Akaike es el de: Demaerschalk (72)")  
} else if ((IATP)<(IAR) && (IATP)<(IAK) && (IATP)<(IAB) && (IATP)<(IAD72)) {
  print ("El modelo mejor ajustado por el indice de Akaike es el de: Thomas Parresol") 
}

#----------------------------------------------------------------------------------

# Indice de Criterio Bayesiano

\begin{equation}\label{eq:ej}
AIC = -2logLik + log(N) * K(6)
\end{equation}

In [ ]:
#?BIC

IBB = BIC (nlmodB)

IBK = BIC (nlmodK)

IBD72 = BIC (nlmodD72) 

IBD73 = BIC (LMD)

IBL = BIC (nlmodL)

IBTP = BIC (modtTP)

IBR = BIC (nlmodR) 
 
IBC = BIC (nlmodC)

IBT = BIC (nlmodT)

#Estructura de control para seleccion del criterio para Bayesiano 

if ((IBR)<(IBK) && (IBR)<(IBB) && (IBR)<(IBD72) && (IBR)<(IBTP)) {
  print ("El modelo mejor ajustado por el indice de Bayesiano es el de: Renteria")
} else if ((IBK)<(IBR) && (IBK)<(IBB) && (IBK)<(IBD72) && (IBR)<(IBTP)) {
  print ("El modelo mejor ajustado por el indice de Bayesiano es el de: Kozak")
} else if ((IBB)<(IBR) && (IBB)<(IBK) && (IBB)<(IBD72)&& (IBR)<(IBTP)) {
  print ("El modelo mejor ajustado por el indice de Bayesiano es el de: Bruce")
} else if ((IBD72)<(IBR) && (IBD72)<(IBK) && (IBD72)<(IBB)&& (IBR)<(IBTP)) {
  print ("El modelo mejor ajustado por el indice de Bayesiano es el de: Demaerschalk (72)")  
} else if ((IBTP)<(IBR) && (IBTP)<(IBK) && (IBTP)<(IBB) && (IBTP)<(IBD72)) {
  print ("El modelo mejor ajustado por el indice de Bayesiano es el de: Thomas y Parresol")
}

# Graficando los perfiles longitudinales fustales

#Que modelos se ajustaron de manera significativa?

#(En este caso):
#1.Demaerschalk (72)
#2.Kozak 
#3.Thomas y Parresol

In [ ]:
#Crear un vector llamado XTZ, con valores que van desde 0 hasta 1 de 0.01 en 0.01  

XTZ <- seq (from = 0, to = 1, by = 0.01)
print(XTZ)

In [ ]:
#1.Reemplazar los coeficientes obtenidos de acuerdo al modelo ajustado (En este caso KOZAK) y encontrar la variable respuesta (y)

summary (nlmodK) 
coe <- coef (nlmodK)
b1 <- coe [1] 
b1
b2 <- coe [2] 
b2

y <- (b1*(XTZ - 1) + b2*(XTZ^2 - 1))
print(y)

win.graph ()
k = plot (XTZ ~ y, data = tabla2, type = "o", lwd = 2, main = "Perfil de Ahusamiento Kozak",
          xlab = "XTZ", ylab = "Y", col = "blue")
grid (col = "black")

In [ ]:
#2.Reemplazar los coeficientes obtenidos de acuerdo al modelo ajustado (En este caso Demaerschalk (72)) y encontrar la variable respuesta (y)

summary (nlmodD72)
coe <- coef (nlmodD72)
b1 <- coe [1]
b1
b2 <- coe [2]
b2

y <- (b1 * (XTZ^b2))
print (y)

win.graph ()
d = plot (XTZ ~ y, data = tabla2, type = "o", lwd = 2, main = "Perfil de Ahusamiento Demaerschalk (72)", 
          xlab = "XTZ", ylab = "Y", col = "red")
grid (col = "blue")

In [ ]:
#3.Reemplazar los coeficientes obtenidos de acuerdo al modelo ajustado (En este caso Thomas y Parresol) y encontrar la variable respuesta (y)

summary (modtTP) 
coe <- coef (modtTP)
b1 <- coe [1] 
b1
b2 <- coe [2] 
b2
b3 <- coe [3] 
b3

y <- ((b1*XTZ-1) + b2*sin(2*pi*XTZ) + b3*(1/tan(pi/2*XTZ))) 
print(y)

win.graph ()
tp = plot (XTZ ~ y, data = tabla2, type = "o", lwd = 2, main = "Perfil de Ahusamiento de Thomas y Parresol", 
           xlab = "XTZ", ylab = "Y", col = "black")
grid (col = "gray")

#                                              Fin del Script